# Импорт нужных библиотек

In [ ]:
import pandas as pd   
from bs4 import BeautifulSoup   
import re 
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

# Загрузим обучающую и тестовую выборку

In [ ]:
train = pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv.zip", header=0, delimiter="\t", quoting=3 )

In [ ]:
train

# Функция по очистке текстов
Функция очищает текст от html тегов и знаков препинания. Затем получаем список всех слов из которого удаляем слова, которые
не влияют на обучение

In [ ]:
def review_to_words( raw_review ):
    review_text = BeautifulSoup(raw_review).get_text()      
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops]   
    return( " ".join( meaningful_words ))   

Обрабатываем все текста в выборке

In [ ]:
num_reviews = train["review"].size
clean_train_reviews = []
for i in range(0, num_reviews):
    if (i%1000 == 0):
        print(i/num_reviews * 100, '%')
    clean_train_reviews.append(review_to_words(train["review"][i] ) )


# Получим вектора призноков из тренировочных данных

In [ ]:
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 20000) 

train_data_features = vectorizer.fit_transform(clean_train_reviews)

train_data_features = train_data_features.toarray()

# Используем RandomForestClassifier в качестве модели

In [ ]:

forest = RandomForestClassifier(n_estimators=110) 
forest = forest.fit( train_data_features, train["sentiment"] )

# Вывод результата
Отчистим тестовые данные и из полученного списка слов получим вектора признаков. Дальше используем обученную модель для получения ответа

In [ ]:
num_reviews = len(test["review"])
clean_test_reviews = [] 

for i in range(0,num_reviews):
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )


test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

result = forest.predict(test_data_features)

output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )